[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PrzemyslawSarnacki/NumericalMethods/blob/master/FloatConverter.ipynb) 

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np 

In [9]:
def list_to_string(list_):
    return "".join([str(elem) for elem in list_])


def decimal_to_binary(decimal):
    binary = []
    while decimal / 2 >= 1:
        i = int(decimal % 2)
        binary.append(i)
        decimal /= 2
    if len(binary) > 1:
        binary.reverse()
    binary.insert(0, 1)
    return binary


def float_to_binary(decimal, place):
    binary = []
    count = 1
    l = len(decimal)
    product = int(decimal)
    while count <= place:
        product = product * (10 ** -l)
        product *= 2
        decimal_int, fraction = str(product).split(".")
        l = len(fraction)
        decimal_int = int(decimal_int)
        fraction = int(fraction)
        binary.append(decimal_int)
        if product == 1.0:
            while len(binary) <= place:
                binary.append(0)
            break
        else:
            product = fraction
        count += 1
    return binary


def binary_to_decimal(binary):
    decimal, i = 0, 0
    while binary != 0:
        dec = binary % 10
        decimal = decimal + dec * 2 ** i
        binary = binary // 10
        i += 1
    return decimal


def treat_bigger_number(num):
    integer, decimal = str(num).split(".")
    integer_string = list(integer)
    sign_check = integer_string.copy()
    if integer_string[0][0] == "-":
        integer_string[0] = integer_string[0][1:]
    integer = list_to_string(integer_string)
    integer = int(integer)

    binary_integer_part = decimal_to_binary(integer)
    exponent = 127 + len((binary_integer_part)) - 1
    binary_exponent = decimal_to_binary(exponent)
    binary_integer_part.pop(0)
    if len(binary_exponent) == 7:
        binary_exponent.insert(0, 0)
    if sign_check[0][0] == "-":
        binary_exponent.insert(0, 1)
    else:
        binary_exponent.insert(0, 0)
    place = 23 - len(binary_integer_part)
    binary_float_part = float_to_binary(decimal, place)
    return binary_exponent, binary_integer_part, binary_float_part


def smaller_than_zero(num):
    count = 0
    while num > -1.0:
        num *= 2
        count += 1
    exponent = 127 - count
    integer, decimal = str(num).split(".")
    integer = int(integer)
    binary_integer_part = decimal_to_binary(integer)
    binary_exponent = decimal_to_binary(exponent)
    binary_integer_part.pop(0)
    if len(binary_exponent) <= 8:
        binary_exponent.insert(0, 0)
    binary_exponent.insert(0, 1)
    place = 23 - len(binary_integer_part)
    binary_float_part = float_to_binary(decimal, place)
    return binary_exponent, binary_integer_part, binary_float_part


def bigger_than_zero(num):
    count = 0
    while num < 1.0:
        num *= 2
        count += 1
    exponent = 127 - count
    integer, decimal = str(num).split(".")
    integer = int(integer)
    binary_integer_part = decimal_to_binary(integer)
    binary_exponent = decimal_to_binary(exponent)
    binary_integer_part.pop(0)
    if len(binary_exponent) <= 8:
        binary_exponent.insert(0, 0)
    binary_exponent.insert(0, 0)
    place = 23 - len(binary_integer_part)
    binary_float_part = float_to_binary(decimal, place)
    return binary_exponent, binary_integer_part, binary_float_part


def ieee_754_to_binary(num):
    if num < 1.0 and num > 0.0:
        binary_exponent, binary_integer_part, binary_float_part = bigger_than_zero(num)
    elif num > -1.0 and num < 0.0:
        binary_exponent, binary_integer_part, binary_float_part = smaller_than_zero(num)
    else:
        binary_exponent, binary_integer_part, binary_float_part = treat_bigger_number(num)

    binary_exponent = list_to_string(binary_exponent)
    binary_integer_part = list_to_string(binary_integer_part)
    binary_float_part = list_to_string(binary_float_part)
    return f"{binary_exponent}{binary_integer_part}{binary_float_part}"


def mantissa_to_float(binary):
    power = -1
    result = 0
    for i in binary:
        result += int(i) * 2 ** (power)
        power -= 1
    return 1 + result


def ieee_754_to_decimal(binary):
    minus = (-1) ** int(binary[0])
    power = binary_to_decimal(int(binary[1:9]))
    mantissa = mantissa_to_float(binary[9::])
    return minus * mantissa * 2 ** (power - 127)

In [3]:
ieee_754_to_binary(-0.1)

'10111101110011001100110011001100'

In [10]:
ieee_754_to_binary(30.14)

'01000001111100010001111010111000'

In [5]:
ieee_754_to_binary(0.5)

'00111111000000000000000000000000'

In [6]:
ieee_754_to_decimal("01000000010010001111010111000010")

3.1399998664855957

In [7]:
ieee_754_to_decimal("10111101110011001100110011001100")

-0.09999999403953552

In [8]:
ieee_754_to_decimal("00111111000000000000000000000000")

0.5